Reference: https://medium.com/rahasak/fine-tuning-llms-on-macos-using-mlx-and-run-with-ollama-182a20f1fd2c

Download the dataset:
`huggingface-cli download gretelai/synthetic_text_to_sql --repo-type dataset`

## Loading and preparing the data

The existing data looks like this: 
```
{
  "id": 39325,
  "domain": "public health",
  "domain_description": "Community health statistics, infectious disease tracking data, healthcare access metrics, and public health policy analysis.",
  "sql_complexity": "aggregation",
  "sql_complexity_description": "aggregation functions (COUNT, SUM, AVG, MIN, MAX, etc.), and HAVING clause",
  "sql_task_type": "analytics and reporting",
  "sql_task_type_description": "generating reports, dashboards, and analytical insights",
  "sql_prompt": "What is the total number of hospital beds in each state?",
  "sql_context": "CREATE TABLE Beds (State VARCHAR(50), Beds INT); INSERT INTO Beds (State, Beds) VALUES ('California', 100000), ('Texas', 85000), ('New York', 70000);",
  "sql": "SELECT State, SUM(Beds) FROM Beds GROUP BY State;",
  "sql_explanation": "This query calculates the total number of hospital beds in each state in the Beds table. It does this by using the SUM function on the Beds column and grouping the results by the State column."
}

```


We will combine `sql_prompt` and `sql_context` into a single prompt field and used the sql field as the completion. Additionally, MLX requires three sets of datasets: train, test, and valid. The data files should be in JSONL format. The final output files will have the following format:
```
{
  "prompt":"What is the total number of tickets sold for all football games? with given SQL schema CREATE TABLE tickets (ticket_id INT, game_id INT, region VARCHAR(50), quantity INT); INSERT INTO tickets (ticket_id, game_id, region, quantity) VALUES (1, 1, 'Midwest', 500); INSERT INTO tickets (ticket_id, game_id, region, quantity) VALUES (2, 2, 'Northeast', 700); CREATE TABLE games (game_id INT, sport VARCHAR(50)); INSERT INTO games (game_id, sport) VALUES (1, 'Football'); INSERT INTO games (game_id, sport) VALUES (2, 'Basketball');",
  "completion":"SELECT SUM(quantity) FROM tickets INNER JOIN games ON tickets.game_id = games.game_id WHERE sport = 'Football';"
}

```

In [ ]:
import pandas as pd

splits = {'train': 'synthetic_text_to_sql_train.snappy.parquet', 'test': 'synthetic_text_to_sql_test.snappy.parquet'}

def prepare_train():
    df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["train"])

    df['prompt'] = df['sql_prompt'] + " with given SQL schema " + df['sql_context']
    df.rename(columns={'sql': 'completion'}, inplace=True)
    df = df[['prompt', 'completion']]

    print(df.head(10))

    # Convert the DataFrame to a JSON format, with each record on a new line
    # save as .jsonl
    df.to_json('train.jsonl', orient='records', lines=True)


def prepare_test_valid():
    df = pd.read_parquet("hf://datasets/gretelai/synthetic_text_to_sql/" + splits["test"])

    df['prompt'] = df['sql_prompt'] + " with given SQL schema " + df['sql_context']
    df.rename(columns={'sql': 'completion'}, inplace=True)
    df = df[['prompt', 'completion']]

    # Calculate split index for two-thirds
    split_index = int(len(df) * 2 / 3)

    # Split the DataFrame into two parts
    test_df = df[:split_index]
    valid_df = df[split_index:]

    print(test_df.head(10))
    print(valid_df.head(10))

    # Save the subsets to their respective JSONL files
    test_df.to_json('test.jsonl', orient='records', lines=True)
    valid_df.to_json('valid.jsonl', orient='records', lines=True)


prepare_train()
prepare_test_valid()

/Users/deyarchit/Projects/ai/ml-training/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                              prompt  \
0  What is the total volume of timber sold by eac...   
1  List all the unique equipment types and their ...   
2  How many marine species are found in the South...   
3  What is the total trade value and average pric...   
4  Find the energy efficiency upgrades with the h...   
5  What is the total spending on humanitarian ass...   
6  What is the average water temperature for each...   
7  Delete a program's outcome data with given SQL...   
8  Find the total fare collected from passengers ...   
9  What is the average property size in inclusive...   

                                          completion  
0  SELECT salesperson_id, name, SUM(volume) as to...  
1  SELECT equipment_type, SUM(maintenance_frequen...  
2  SELECT COUNT(*) FROM marine_species WHERE loca...  
3  SELECT trader_id, stock, SUM(price * quantity)...  
4  SELECT type, cost FROM (SELECT type, cost, ROW...  
5  SELECT SUM(spending) FROM defense.eu_humanitar... 